In [1]:
import torch
from torch import nn

In [2]:
torch.__version__

'1.12.1+cu102'

In [86]:
class ConvLayers(nn.Module):
    """ 
        Conv layers with batchnorm and activation layers
    """
    def __init__(self, input_ch, output_ch, kernel_size, stride, padding=1):
        super(ConvLayers, self).__init__()
        if kernel_size == 1:
            self.conv_layer = nn.Conv2d(input_ch, output_ch, kernel_size=kernel_size, stride=stride)
        else:
            self.conv_layer = nn.Conv2d(input_ch, output_ch, kernel_size=kernel_size, stride=stride, padding=padding)
        self.norm = nn.BatchNorm2d(output_ch)
        self.act = nn.GELU()

    def forward(self, x):
        return self.act(self.norm(self.conv_layer(x)))


class ConvLayers_1(nn.Module):
    """ 
        Conv layers with batchnorm and activation layers
    """
    def __init__(self, input_ch, output_ch, kernel_size, stride):
        super(ConvLayers_1, self).__init__()
        if stride == 2:
            self.conv_layer = nn.Conv2d(input_ch, output_ch, kernel_size=kernel_size, stride=stride)
        else:
            self.conv_layer = nn.Conv2d(input_ch, output_ch, kernel_size=kernel_size, stride=stride, padding="same")
        self.norm = nn.BatchNorm2d(output_ch)
        self.act = nn.GELU()

    def forward(self, x):
        return self.act(self.norm(self.conv_layer(x)))


class ConvBlock(nn.Module):
    """ Conv block with residual connections 
    """
    def __init__(self, list_conv_layers, num_repeat):
        super(ConvBlock, self).__init__()
        self.repeat = num_repeat
        temp_conv_layers = []
        for layer in list_conv_layers:
            temp_conv_layers.append(ConvLayers(layer[0], layer[1], layer[2], layer[3]))
        self.conv_layers = nn.Sequential(*temp_conv_layers)

    def forward(self, x):
        x_ = x
        for _ in range(self.repeat):
            x = self.conv_layers(x)
        return x + x_


class DarkNetHead(nn.Module):

    def __init__(self, model_parameters):
        super(DarkNetHead, self).__init__()
        temp_layers = []
        for layers in model_parameters:
            if layers[0] == "cl":
                temp_layers.append(ConvLayers_1(layers[1], layers[2], layers[3], layers[4]))
            elif layers[0] == "cb":
                temp_layers.append(ConvBlock(layers[1], layers[2]))
        self.conv_layers = nn.Sequential(*temp_layers)

    def forward(self, x):
        return self.conv_layers(x)




model_parameters = [
    ("cl", 3, 32, 3, 1),
    ("cl", 32, 64, 3, 2),
    ("cb", [(64, 32, 1, 1), (32, 64, 3, 1)], 1),
    ("cl", 64, 128, 3, 2),
    ("cb", [(128, 64, 1, 1), (64, 128, 3, 1)], 2),
    ("cl", 128, 256, 3, 2),
    ("cb", [(256, 128, 1, 1), (128, 256, 3, 1)], 8),
    ("cl", 256, 512, 3, 2),
    ("cb", [(512, 256, 1, 1), (256, 512, 3, 1)], 8),
    ("cl", 512, 1024, 3, 2),
    ("cb", [(1024, 512, 1, 1), (512, 1024, 3, 1)], 4)
]



In [87]:
conv1 = ConvLayers_1(64, 32, 1, 1)
conv2 = ConvLayers_1(32, 64, 7, 1)
inp = torch.rand((4, 64, 256, 256))
inp1 = conv1(inp)
print(inp1.shape)
inp2 = conv2(inp1)
print(inp2.shape)

torch.Size([4, 32, 256, 256])
torch.Size([4, 64, 256, 256])


In [88]:
inp = torch.rand((4, 3, 256, 256))

In [89]:
model = DarkNetHead(model_parameters)

In [90]:
out = model(inp)
out.shape

torch.Size([4, 1024, 7, 7])

In [ ]:
p1 = 48
p2 = 72
p3 = 96
p4 = 6

pool w 

p1  48 = w 
p2 

In [ ]:
p1 = w/48

p2 w/72
p3 w/96

p4 w/6

w/()

In [94]:

288/6

48.0

In [95]:
(6 + 4 + 3 + 48)/288

0.21180555555555555

In [96]:
288/(6+4+3+48)

4.721311475409836

In [97]:
288/61

4.721311475409836